<a href="https://colab.research.google.com/github/ethansong206/Climate-Plus-Project/blob/main/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import necessary packages for the code

In [2]:
import pandas as pd
import numpy as np

Load all the csv files in and break up the data by

1) Location and

2) Year

In [27]:
DiningDataFull = pd.read_csv('Climate+ Data 2019 thru 2023.csv')
#print(DiningDataFull['Unit Name'].value_counts()) # Show how many entries there are of each location

#below code to make 'Unit Name' column easier to handle
#can add more lines given more locations
def location_rename(location):
    if("Marketplace" in location):
        return "Marketplace"
    if("Marine Lab" in location):
        return "DuML"
    if("Trinity" in location):
        return "Trinity"
    if("Freeman" in location):
        return "Freeman"
    return None

DiningDataFull['Unit Name'] = DiningDataFull.apply(lambda m: location_rename(m['Unit Name']), axis = 1)

print(DiningDataFull.head())

<ipython-input-27-efd2f7e53b18>:1: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  DiningDataFull = pd.read_csv('Climate+ Data 2019 thru 2023.csv')


     Unit Name    Vendor Purchase Date Vendor Item ID  \
0  Marketplace  US Foods      7/1/2019        5018999   
1  Marketplace  US Foods      7/1/2019        5400692   
2  Marketplace  US Foods      7/1/2019        8012643   
3  Marketplace  US Foods      7/1/2019         731349   
4  Marketplace  US Foods      7/1/2019          68106   

                             Vendor Item Description  \
0  POTATO, FRENCH-FRY 3/4 X3/8 STEAK CUT SKIN-ON ...   
1  POTATO, FRENCH-FRY SPIRAL COATED SEASONED TFF ...   
2  POTATO, FRENCH-FRY 1/2 CRINKLE-CUT TFF EXTRA-L...   
3                POTATO, HASH BROWN SHRED COOKED REF   
4    POTATO, TATER NUGGET PARFRIED FROZEN HASH BROWN   

  Vendor Item Purchase Unit                              Product Group  \
0                    6/5 LB  APPETIZERS, ENTREES, & POTATOES REF & FZN   
1                    6/5 LB  APPETIZERS, ENTREES, & POTATOES REF & FZN   
2                  6/4.5 LB  APPETIZERS, ENTREES, & POTATOES REF & FZN   
3                   2/10

Next code chunk will be used to convert all units to the equivalent amount in grams since our data on carbon emissions is all in the unit of grams